# Demostración

In [1]:
import pandas as pd
import numpy as np
import pickle
import time
import datetime
from tensorflow import keras

In [2]:
model = keras.models.load_model("modelo_pronosticador_regina.h5")

file = open("norm.params", "rb")
train_mean, train_std, wind_dic = pickle.load(file)
file.close()

In [3]:
drop = [0, 2, 3, 11, 12, 14, 13, 8, 19, 20, 26, 30, 31]

day = 24*60*60
year = 365.2425 * day

def convert(data):
    data = data.splitlines()
    conv_data = np.zeros((0, 23))
    
    i = 0
    
    for sample in data:
        sample = sample.split("\t")
        date = sample[0]
        sample[7]  = wind_dic[sample[7]]
        sample[10] = wind_dic[sample[10]]
        sample = np.delete(sample, drop)
        sample = sample.astype(float)
        sample[0] -= 2
        
        date = pd.Timestamp.timestamp(pd.Timestamp(datetime.datetime.strptime(date, "%d.%m.%y %H:%M")))
        
        day_sin  = np.sin(date * (2 * np.pi / day))
        day_cos  = np.cos(date * (2 * np.pi / day))
        year_sin = np.sin(date * (2 * np.pi / year))
        year_cos = np.cos(date * (2 * np.pi / year))
        
        sample = np.concatenate((sample, [day_sin, day_cos, year_sin, year_cos]))
        
        sample = (sample-train_mean)/train_std
        
        conv_data = np.vstack((conv_data, sample))
    
    conv_data = np.reshape(conv_data, (1, 90, 23))
    return conv_data

In [7]:
x = convert("""""")

pred = model.predict(x)

print(pred)

[[0.4756378]]
